In [1]:
import pandas as pd
import yfinance as yf
import os
import pytz

# New York Zeitzone
NY_TZ = pytz.timezone("America/New_York")

def calculate_indicators(data):
    """Berechnet technische Indikatoren für die Aktienkursdaten."""

    # RSI-Berechnung
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # EMA
    data['EMA9'] = data['Close'].ewm(span=9, adjust=False).mean()
    data['EMA21'] = data['Close'].ewm(span=21, adjust=False).mean()

    # Bollinger Bänder
    sma20 = data['Close'].rolling(window=20).mean()
    rolling_std = data['Close'].rolling(window=20).std()
    data['Upper Band'] = sma20 + (2 * rolling_std)
    data['Lower Band'] = sma20 - (2 * rolling_std)

    # ATR
    data['High-Low'] = data['High'] - data['Low']
    data['High-PrevClose'] = abs(data['High'] - data['Close'].shift(1))
    data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=14).mean()

    # # Sicherstellen, dass alle Werte numerisch sind
    # cols_to_fix = ['High', 'Low', 'Close', 'Volume']
    # for col in cols_to_fix:
    #     data[col] = pd.to_numeric(data[col], errors='coerce')

    # # Fehlende Werte durch 0 ersetzen
    # data.fillna({'High': 0, 'Low': 0, 'Close': 0, 'Volume': 0}, inplace=True)

    # # **VWAP Berechnung**
    # data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
    # data['Typical_Price'] = data['Typical_Price'].fillna(0)
    # data['Volume'] = data['Volume'].fillna(0)

    # data['Cum_TPxV'] = (data['Typical_Price'] * data['Volume']).cumsum()
    # data['Cum_Volume'] = data['Volume'].cumsum()

    # # Schutz gegen Division durch Null
    # data['VWAP'] = data['Cum_TPxV'] / data['Cum_Volume']
    # data.loc[data['Cum_Volume'] == 0, 'VWAP'] = None 

    # #  Debug-Ausgabe für VWAP
    # print(" VWAP Debug-Daten:")
    # print(data[['Typical_Price', 'Volume', 'Cum_TPxV', 'Cum_Volume', 'VWAP']].head(20))




    # Stochastic Oscillator
    low_min = data['Low'].rolling(window=14).min()
    high_max = data['High'].rolling(window=14).max()
    data['Stochastic'] = 100 * ((data['Close'] - low_min) / (high_max - low_min))

    # MACD
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['Signal Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Prozentuale Änderung
    data['Pct_Change'] = data['Close'].pct_change()
    
    data.head(10)

    return data

def fetch_and_save_stock_data(ticker, start_date, end_date, interval, save_path):
    """
    Lädt Aktienkursdaten und speichert sie als CSV.

    Args:
        ticker (str): Aktien-Ticker (z.B. 'AAPL').
        start_date (str): Startdatum ('YYYY-MM-DD').
        end_date (str): Enddatum ('YYYY-MM-DD').
        interval (str): Intervall ('1h', '1d', etc.).
        save_path (str): Speicherpfad.
    """
    try:
        print(f"Abrufe Daten für {ticker} von {start_date} bis {end_date} mit Intervall {interval}...")

        # 📌 Daten abrufen
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)

        if data.empty:
            print(f"❌ Keine Daten für {ticker} gefunden.")
            return

        # 📌 Index zurücksetzen (damit "Date" als Spalte erscheint)
        data.reset_index(inplace=True)

        # 📌 Zeitzone auf New York setzen
        data['Datetime'] = data['Datetime'].dt.tz_convert(NY_TZ).dt.tz_localize(None)

        # 📌 Indikatoren berechnen
        data = calculate_indicators(data)

        # 📌 Speichern als CSV
        os.makedirs(save_path, exist_ok=True)
        file_name = f"{ticker}.csv"
        file_path = os.path.join(save_path, file_name)

        # Datei überschreiben (keine Erweiterung)
        data.to_csv(file_path, index=False)
        print(f"✅ Datei gespeichert: {file_path}")

    except Exception as e:
        print(f"⚠️ Fehler für {ticker}: {e}")

def main():
    # Aktienliste & Parameter
    stocks = ['AAL','AAPL','ANET','BA', 'BYND','DAL','DELL', 'FTNT','IBM','LUV', 'NFLX','NVDA','OXY','PDD','PYPL','RIO','SPCE','TSLA','UAL','MSFT', 'GOOGL', 'META']
    start_date = '2023-03-01'
    end_date = '2024-12-31'
    interval = '1h'
    save_path = "C:/Users/Martin/Documents/StockData"

    # Daten abrufen & speichern
    for stock in stocks:
        fetch_and_save_stock_data(stock, start_date, end_date, interval, save_path)
        

if __name__ == "__main__":
    main()


Abrufe Daten für AAL von 2023-03-01 bis 2024-12-31 mit Intervall 1h...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


❌ Keine Daten für AAL gefunden.
Abrufe Daten für AAPL von 2023-03-01 bis 2024-12-31 mit Intervall 1h...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


❌ Keine Daten für AAPL gefunden.
Abrufe Daten für ANET von 2023-03-01 bis 2024-12-31 mit Intervall 1h...


...

❌ Keine Daten für MSFT gefunden.
Abrufe Daten für GOOGL von 2023-03-01 bis 2024-12-31 mit Intervall 1h...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GOOGL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


❌ Keine Daten für GOOGL gefunden.
Abrufe Daten für META von 2023-03-01 bis 2024-12-31 mit Intervall 1h...


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['META']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


❌ Keine Daten für META gefunden.
